In [1]:
# 自定义resnet模型,使用imagenet的参数
import numpy as np
import tensorflow as tf
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import Model, load_model
import os, cv2, random
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.callbacks import Callback, EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score
from keras.applications.resnet50 import ResNet50
from keras import optimizers
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation,BatchNormalization,GlobalAveragePooling2D

Using TensorFlow backend.


In [2]:
# 创建相关文件夹
DATA_DIR = './data/'  # 数据集路径
OUTPUT = './output' # 模型保存位置
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)
    
# 下载数据集
from modelarts.session import Session
session = Session()

session.download_data(
    bucket_path="modelarts-labs/notebook/DL_image_recognition/image_recognition.tar.gz",
    path="./image_recognition.tar.gz")

# 使用tar命令解压资源包
!tar xf ./image_recognition.tar.gz

# 清理压缩包
!rm -f ./image_recognition.tar.gz



ROWS = 225
COLS = 225
CHANNELS = 3

Successfully download file modelarts-labs/notebook/DL_image_recognition/image_recognition.tar.gz from OBS to local ./image_recognition.tar.gz


In [3]:
image_file_names = [DATA_DIR + i for i in os.listdir(DATA_DIR)]
dogs = [DATA_DIR + i for i in os.listdir(DATA_DIR) if 'dog.' in i]
cats = [DATA_DIR + i for i in os.listdir(DATA_DIR) if 'cat.' in i]

# 数据洗牌
random.shuffle(image_file_names)


def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)  # 彩色模式读取图像
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(image_file_names):
    count = len(image_file_names)
    data = np.ndarray((count, ROWS, COLS, CHANNELS), dtype=np.uint8)

    for i, image_file in enumerate(image_file_names):
        image = read_image(image_file)
        data[i] = image
        if i % 1000 == 0: print('Processed {} of {}'.format(i, count))

    return data


# 读取图片至内存
images = prep_data(image_file_names)
# 我们根据图片名称包含的字符串给图片打标签。我们用0表示cat，1表示dog。
labels = []
for i in image_file_names:
    if 'dog.' in i:
        labels.append(1)
    else:
        labels.append(0)
# 将标签做热点处理
labels = np_utils.to_categorical(labels, 2)
# 切分数据集
train_data, test_data, train_label, test_label = train_test_split(images, labels, test_size=0.25, random_state=10)
train_data, validate_data, train_label, validate_label = train_test_split(train_data, train_label, test_size=0.25, random_state=10)

Processed 0 of 25000
Processed 1000 of 25000
Processed 2000 of 25000
Processed 3000 of 25000
Processed 4000 of 25000
Processed 5000 of 25000
Processed 6000 of 25000
Processed 7000 of 25000
Processed 8000 of 25000
Processed 9000 of 25000
Processed 10000 of 25000
Processed 11000 of 25000
Processed 12000 of 25000
Processed 13000 of 25000
Processed 14000 of 25000
Processed 15000 of 25000
Processed 16000 of 25000
Processed 17000 of 25000
Processed 18000 of 25000
Processed 19000 of 25000
Processed 20000 of 25000
Processed 21000 of 25000
Processed 22000 of 25000
Processed 23000 of 25000
Processed 24000 of 25000


In [4]:
epochs = 100  # 训练轮数
batch_size = 16  # 批大小
# Loss记录
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
def run_catdog():
    history = LossHistory()
    # 开始训练
    history=model.fit(train_data, train_label,
              epochs=epochs, batch_size=batch_size,
              verbose=2,shuffle=True,validation_data=(validate_data,validate_label),callbacks=[history, early_stopping])
    return history

In [5]:
base_model = ResNet50(weights="imagenet", include_top=False, pooling=None, input_shape=(ROWS, COLS, CHANNELS), classes=2)
# 冻结base_model所有层，这样就可以正确获得bottleneck特征
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
#添加自己的全链接分类层
x = Flatten()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)
#训练模型
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

94658560/94653016 [==============================] - 116s 1us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 225, 225, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 231, 231, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 113, 113, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 113, 113, 64) 256         conv1[0][0]                      
__________________________________________

In [6]:
optimizer = optimizers.RMSprop(lr=1e-4)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

history = run_catdog()
# 训练好的模型可以保存起来，永久使用

model.save(os.path.join(OUTPUT, 'model.h5'))

Train on 14062 samples, validate on 4688 samples
Epoch 1/100
 - 78s - loss: 0.6890 - acc: 0.9445 - val_loss: 0.4423 - val_acc: 0.9674
Epoch 2/100
 - 71s - loss: 0.5266 - acc: 0.9600 - val_loss: 0.3349 - val_acc: 0.9748
Epoch 3/100
 - 70s - loss: 0.4871 - acc: 0.9643 - val_loss: 0.4895 - val_acc: 0.9665
Epoch 4/100
 - 71s - loss: 0.4207 - acc: 0.9701 - val_loss: 0.3335 - val_acc: 0.9763
Epoch 5/100
 - 71s - loss: 0.3891 - acc: 0.9723 - val_loss: 0.4649 - val_acc: 0.9689
Epoch 6/100
 - 70s - loss: 0.3546 - acc: 0.9748 - val_loss: 0.5674 - val_acc: 0.9610
Epoch 7/100
 - 71s - loss: 0.3376 - acc: 0.9761 - val_loss: 0.5106 - val_acc: 0.9654
Epoch 00007: early stopping


In [7]:
# 预测测试集
predictions = model.predict(test_data, verbose=0)
# 评估模型准确度
predictions_test = []
test_label_test = []
for p in predictions:

    if int(round(p[0])) == 1:
        predictions_test.append(1)
    else:
        predictions_test.append(0)
for t in test_label:
    if int(t[0]) == 1:
        test_label_test.append(1)
    else:
        test_label_test.append(0)
acc = accuracy_score(test_label_test, predictions_test)

print('训练得到的猫狗识别模型的准确度是：%f' % acc)

训练得到的猫狗识别模型的准确度是：0.963680
